Original code from Jurgens Fourie, used for this study with minor adjustments. For the study by Fourie et al., please see: Fourie, J., Klingwort, J., and Gootzen. Y (2025). Rule-based transport mode classification in a smartphone-based travel survey. CBS Discussion Paper. 

In [ ]:
from pyrosm import get_data
from pyrosm import OSM
import pandas as pd
from shapely.wkt import loads
import numpy as np
from shapely.geometry import LineString, Point
import geopandas as gpd
import time
from multiprocessing import Pool
from shapely import wkt
import warnings

# Get OSM points of interest (POI's) for regions in Germany

In [ ]:
osm_ns = OSM("niedersachsen-latest.osm.pbf")
osm_bw = OSM("baden-wuerttemberg-latest.osm.pbf")
osm_nw = OSM("nordrhein-westfalen-latest.osm.pbf")

#### Train routes

In [ ]:
# Create a custom filter to fetch only public transport nodes for trains
custom_filter = {'type': ['network'],
                "route": ['train']
                }

# Read points of interest (POIs) with the custom filter
train_routes_ns = osm_ns.get_pois(custom_filter=custom_filter)
train_routes_bw = osm_bw.get_pois(custom_filter=custom_filter)
train_routes_nw = osm_nw.get_pois(custom_filter=custom_filter)

# keep only route data
train_routes_ns = train_routes_ns[train_routes_ns['type']=='route']
train_routes_bw = train_routes_bw[train_routes_bw['type']=='route']
train_routes_nw = train_routes_nw[train_routes_nw['type']=='route']

# Write the dataframe to a CSV file
train_routes_ns.to_csv("data/osm/poi/osm_poi_train_routes_ns.csv", index=False)
train_routes_bw.to_csv("data/osm/poi/osm_poi_train_routes_bw.csv", index=False)
train_routes_nw.to_csv("data/osm/poi/osm_poi_train_routes_nw.csv", index=False)

#### Tram routes

In [ ]:
custom_filter = {'type': ['network'],
                "route": ['tram']
                }

tram_routes_ns = osm_ns.get_pois(custom_filter=custom_filter)
tram_routes_bw = osm_bw.get_pois(custom_filter=custom_filter)
tram_routes_nw = osm_nw.get_pois(custom_filter=custom_filter)

tram_routes_ns = tram_routes_ns[tram_routes_ns['type']=='route']
tram_routes_bw = tram_routes_bw[tram_routes_bw['type']=='route']
tram_routes_nw = tram_routes_nw[tram_routes_nw['type']=='route']

tram_routes_ns.to_csv("data/osm/poi/osm_poi_tram_routes_ns.csv", index=False)
tram_routes_bw.to_csv("data/osm/poi/osm_poi_tram_routes_bw.csv", index=False)
tram_routes_nw.to_csv("data/osm/poi/osm_poi_tram_routes_nw.csv", index=False)

#### Bus routes

In [ ]:
custom_filter = {'type': ['network'],
                "route": ['bus']
                }

bus_routes_ns = osm_ns.get_pois(custom_filter=custom_filter)
bus_routes_bw = osm_bw.get_pois(custom_filter=custom_filter)
bus_routes_nw = osm_nw.get_pois(custom_filter=custom_filter)

bus_routes_ns = bus_routes_ns[bus_routes_ns['type']=='route']
bus_routes_bw = bus_routes_bw[bus_routes_bw['type']=='route']
bus_routes_nw = bus_routes_nw[bus_routes_nw['type']=='route']

bus_routes_ns.to_csv("data/osm/poi/osm_poi_bus_routes_ns.csv", index=False)
bus_routes_bw.to_csv("data/osm/poi/osm_poi_bus_routes_bw.csv", index=False)
bus_routes_nw.to_csv("data/osm/poi/osm_poi_bus_routes_nw.csv", index=False)

#### Metro routes

In [ ]:
# subway routes are only in niedersachsen-westfalen
custom_filter = {'type': ['network'],
                "route": ['subway']
                }

# metro_routes_ns = osm_ns.get_pois(custom_filter=custom_filter)
# metro_routes_bw = osm_bw.get_pois(custom_filter=custom_filter)
metro_routes_nw = osm_nw.get_pois(custom_filter=custom_filter)

# metro_routes_ns = metro_routes_ns[metro_routes_ns['type']=='route']
# metro_routes_bw = metro_routes_bw[metro_routes_bw['type']=='route']
metro_routes_nw = metro_routes_nw[metro_routes_nw['type']=='route']

# metro_routes_ns.to_csv("osm_poi_metro_routes_ns.csv", index=False)
# metro_routes_bw.to_csv("osm_poi_metro_routes_bw.csv", index=False)
metro_routes_nw.to_csv("data/osm/poi/osm_poi_metro_routes_nw.csv", index=False)

In [ ]:
# subway routes are only in niedersachsen-westfalen
custom_filter = {'type': ['network'],
                "route": ['subway']
                }

metro_routes_ns = osm_ns.get_pois(custom_filter=custom_filter)

#### Bike routes

In [ ]:
custom_filter = {'type': ['network'],
                "route": ['bicycle']
                }

bike_routes_ns = osm_ns.get_pois(custom_filter=custom_filter)
bike_routes_bw = osm_bw.get_pois(custom_filter=custom_filter)
bike_routes_nw = osm_nw.get_pois(custom_filter=custom_filter)

bike_routes_ns = bike_routes_ns[bike_routes_ns['type']=='route']
bike_routes_bw = bike_routes_bw[bike_routes_bw['type']=='route']
bike_routes_nw = bike_routes_nw[bike_routes_nw['type']=='route']

bike_routes_ns.to_csv("data/osm/poi/osm_poi_bike_routes_ns.csv", index=False)
bike_routes_bw.to_csv("data/osm/poi/osm_poi_bike_routes_bw.csv", index=False)
bike_routes_nw.to_csv("data/osm/poi/osm_poi_bike_routes_nw.csv", index=False)

#### Train stops

In [ ]:
custom_filter = {'public_transport': ['stop_position'],
                "railway": ['station','halt']
                }

train_stops_ns = osm_ns.get_pois(custom_filter=custom_filter)
train_stops_bw = osm_bw.get_pois(custom_filter=custom_filter)
train_stops_nw = osm_nw.get_pois(custom_filter=custom_filter)

train_stops_ns = train_stops_ns[train_stops_ns['osm_type'] == 'node']
train_stops_bw = train_stops_bw[train_stops_bw['osm_type'] == 'node']
train_stops_bw = train_stops_bw[train_stops_bw['osm_type'] == 'node']

# Filter the DataFrame to include only specific values in the 'railway' column
train_stops_ns = train_stops_ns[train_stops_ns['railway'].isin(['stop', 'station'])]
train_stops_bw = train_stops_bw[train_stops_bw['railway'].isin(['stop', 'station'])]
train_stops_nw = train_stops_nw[train_stops_nw['railway'].isin(['stop', 'station'])]

train_stops_ns.to_csv("data/osm/poi/osm_poi_train_stops_ns.csv", index=False)
train_stops_bw.to_csv("data/osm/poi/osm_poi_train_stops_bw.csv", index=False)
train_stops_nw.to_csv("data/osm/poi/osm_poi_train_stops_nw.csv", index=False)

#### Tram stops

In [ ]:
custom_filter_tram = { 
    "railway": ["tram_stop"]  # Specify tram as the railway type
}

tram_stops_ns = osm_ns.get_pois(custom_filter=custom_filter_tram)
tram_stops_bw = osm_bw.get_pois(custom_filter=custom_filter_tram)
tram_stops_nw = osm_nw.get_pois(custom_filter=custom_filter_tram)

tram_stops_ns = tram_stops_ns[tram_stops_ns['osm_type'] == 'node']
tram_stops_bw = tram_stops_bw[tram_stops_bw['osm_type'] == 'node']
tram_stops_nw = tram_stops_nw[tram_stops_nw['osm_type'] == 'node']

tram_stops_ns.to_csv("data/osm/poi/osm_poi_tram_stops_ns.csv", index=False)
tram_stops_bw.to_csv("data/osm/poi/osm_poi_tram_stops_bw.csv", index=False)
tram_stops_nw.to_csv("data/osm/poi/osm_poi_tram_stops_nw.csv", index=False)

#### Bus stops

In [ ]:
custom_filter_bus = {
    "highway": ["bus_stop"]  # Specify bus as the railway type
}

bus_stops_ns = osm_ns.get_pois(custom_filter=custom_filter_bus)
bus_stops_bw = osm_bw.get_pois(custom_filter=custom_filter_bus)
bus_stops_nw = osm_nw.get_pois(custom_filter=custom_filter_bus)

bus_stops_ns = bus_stops_ns[bus_stops_ns['osm_type'] == 'node']
bus_stops_bw = bus_stops_bw[bus_stops_bw['osm_type'] == 'node']
bus_stops_nw = bus_stops_nw[bus_stops_nw['osm_type'] == 'node']

bus_stops_ns.to_csv("data/osm/poi/osm_poi_bus_stops_ns.csv", index=False)
bus_stops_bw.to_csv("data/osm/poi/osm_poi_bus_stops_bw.csv", index=False)
bus_stops_nw.to_csv("data/osm/poi/osm_poi_bus_stops_nw.csv", index=False)

#### Metro stops

In [ ]:
custom_filter_metro = {
    "railway": ["station"],
    "station": ['subway']
}

metro_stops_ns = osm_ns.get_pois(custom_filter=custom_filter_metro)
metro_stops_bw = osm_bw.get_pois(custom_filter=custom_filter_metro)
metro_stops_nw = osm_nw.get_pois(custom_filter=custom_filter_metro)

metro_stops_ns = metro_stops_ns[metro_stops_ns['osm_type'] == 'node']
metro_stops_bw = metro_stops_bw[metro_stops_bw['osm_type'] == 'node']
metro_stops_nw = metro_stops_nw[metro_stops_nw['osm_type'] == 'node']

metro_stops_ns.to_csv("data/osm/poi/osm_poi_metro_stops_ns.csv", index=False)
metro_stops_bw.to_csv("data/osm/poi/osm_poi_metro_stops_bw.csv", index=False)
metro_stops_nw.to_csv("data/osm/poi/osm_poi_metro_stops_nw.csv", index=False)

#### Parking amenities

In [ ]:
custom_filter = {
    'amenity': ['parking', 'bicycle_parking', 
                'parking_entrance', 'parking_underground', 
                'bicycle_rental', 'fuel', 'charging_station']
}

parking_amenities_ns = osm_ns.get_pois(custom_filter=custom_filter)
parking_amenities_bw = osm_bw.get_pois(custom_filter=custom_filter)
parking_amenities_nw = osm_nw.get_pois(custom_filter=custom_filter)

parking_amenities_ns.to_csv("data/osm/poi/osm_poi_parking_amenities_ns.csv", index=False)
parking_amenities_bw.to_csv("data/osm/poi/osm_poi_parking_amenities_bw.csv", index=False)
parking_amenities_nw.to_csv("data/osm/poi/osm_poi_parking_amenities_nw.csv", index=False)

#### Traffic indicators

In [ ]:
custom_filter = {'type': ['traffic'],
                "highway": ['traffic_signals','mini_roundabout',
                            'stop', 'crossing', 'level_crossing',
                            'ford', 'motorway_junction', 'turning_circle',
                            'speed_camera', 'street_lamp']
                }

traffic_indicators_ns = osm_ns.get_pois(custom_filter=custom_filter)
traffic_indicators_bw = osm_bw.get_pois(custom_filter=custom_filter)
traffic_indicators_nw = osm_nw.get_pois(custom_filter=custom_filter)

traffic_indicators_ns.to_csv("data/osm/poi/osm_poi_traffic_indicators_ns.csv", index=False)
traffic_indicators_bw.to_csv("data/osm/poi/osm_poi_traffic_indicators_bw.csv", index=False)
traffic_indicators_nw.to_csv("data/osm/poi/osm_poi_traffic_indicators_nw.csv", index=False)

# OSM data preparation

#### Load OSM data from NL

In [ ]:
train_routes_nl = pd.read_csv("data/osm/poi/osm_poi_train_routes.csv")
metro_routes_nl = pd.read_csv("data/osm/poi/osm_poi_metro_routes.csv")
bus_routes_nl = pd.read_csv(".data/osm/poi/osm_poi_bus_routes.csv")
bike_routes_nl = pd.read_csv("data/osm/poi/osm_poi_bike_routes.csv")
tram_routes_nl = pd.read_csv("data/osm/poi/osm_poi_tram_routes.csv")

train_stops_nl = pd.read_csv('data/osm/poi/osm_poi_train_stops.csv')
tram_stops_nl = pd.read_csv('data/osm/poi/osm_poi_tram_stops.csv')
metro_stops_nl = pd.read_csv('data/osm/poi/osm_poi_metro_stops.csv')
bus_stops_nl = pd.read_csv('data/osm/poi/osm_poi_bus_stops.csv')

parking_amenities_nl = pd.read_csv('data/osm/poi/osm_poi_parking_amenities.csv')

traffic_indicators_nl = pd.read_csv(".data/osm/poi/osm_poi_traffic_indicators.csv")

#### Load OSM data from Germany (Niedersachsen, Baden-Wuertemberg, Nordrhein-Westfalen)

In [ ]:
train_routes_ns = pd.read_csv("data/osm/poi/osm_poi_train_routes_ns.csv")

metro_routes_ns = pd.read_csv("data/osm/poi/osm_poi_metro_routes_ns.csv")
bus_routes_ns = pd.read_csv("data/osm/poi/osm_poi_bus_routes_ns.csv")
bike_routes_ns = pd.read_csv("data/osm/poi/osm_poi_bike_routes_ns.csv")
tram_routes_ns = pd.read_csv("data/osm/poi/osm_poi_tram_routes_ns.csv")

train_stops_ns = pd.read_csv('data/osm/poi/osm_poi_train_stops_ns.csv')
tram_stops_ns = pd.read_csv('data/osm/poi/osm_poi_tram_stops_ns.csv')
metro_stops_ns = pd.read_csv('data/osm/poi/osm_poi_metro_stops_ns.csv')
bus_stops_ns = pd.read_csv('data/osm/poi/osm_poi_bus_stops_ns.csv')

parking_amenities_ns = pd.read_csv('data/osm/poi/osm_poi_parking_amenities_ns.csv')

traffic_indicators_ns = pd.read_csv("data/osm/poi/osm_poi_traffic_indicators_ns.csv")

In [ ]:
train_routes_bw = pd.read_csv(".data/osm/poi/osm_poi_train_routes_bw.csv")

metro_routes_bw = pd.read_csv(".data/osm/poi/osm_poi_metro_routes_bw.csv")
bus_routes_bw = pd.read_csv("data/osm/poi/osm_poi_bus_routes_bw.csv")
bike_routes_bw = pd.read_csv("data/osm/poi/osm_poi_bike_routes_bw.csv")
tram_routes_bw = pd.read_csv(".data/osm/poi/osm_poi_tram_routes_bw.csv")

train_stops_bw = pd.read_csv('data/osm/poi/osm_poi_train_stops_bw.csv')
tram_stops_bw = pd.read_csv('data/osm/poi/osm_poi_tram_stops_bw.csv')
metro_stops_bw = pd.read_csv('dataosm/poi/osm_poi_metro_stops_bw.csv')
bus_stops_bw = pd.read_csv('data/osm/poi/osm_poi_bus_stops_bw.csv')

parking_amenities_bw = pd.read_csv('data/osm/poi/osm_poi_parking_amenities_bw.csv')

traffic_indicators_bw = pd.read_csv("data/osm/poi/osm_poi_traffic_indicators_bw.csv")

In [ ]:
train_routes_nw = pd.read_csv("data/osm/poi/osm_poi_train_routes_ns.csv")
metro_routes_nw = pd.read_csv("data/osm/poi/osm_poi_metro_routes_nw.csv")
bus_routes_nw = pd.read_csv("data/osm/poi/osm_poi_bus_routes_nw.csv")
bike_routes_nw = pd.read_csv("data/osm/poi/osm_poi_bike_routes_nw.csv")
tram_routes_nw = pd.read_csv("data/osm/poi/osm_poi_tram_routes_nw.csv")

train_stops_nw = pd.read_csv('data/osm/poi/osm_poi_train_stops_nw.csv')
tram_stops_nw = pd.read_csv('data/osm/poi/osm_poi_tram_stops_nw.csv')
metro_stops_nw = pd.read_csv('data/osm/poi/osm_poi_metro_stops_nw.csv')
bus_stops_nw = pd.read_csv('data/osm/poi/osm_poi_bus_stops_nw.csv')

parking_amenities_nw = pd.read_csv('data/osm/poi/osm_poi_parking_amenities_nw.csv')

traffic_indicators_nw = pd.read_csv("data/osm/poi/osm_poi_traffic_indicators_nw.csv")

#### Combine different regions/countries

In [ ]:
train_routes = pd.concat([train_routes_nl, train_routes_ns, train_routes_bw, train_routes_nw],
                         ignore_index=True)
metro_routes = pd.concat([metro_routes_nl, metro_routes_ns, metro_routes_bw, metro_routes_nw],
                         ignore_index=True)
bus_routes = pd.concat([bus_routes_nl, bus_routes_ns, bus_routes_bw, bus_routes_nw],
                       ignore_index = True)
bike_routes = pd.concat([bike_routes_nl, bike_routes_ns, bike_routes_bw, bike_routes_nw],
                       ignore_index = True)
tram_routes = pd.concat([tram_routes_nl, tram_routes_ns, tram_routes_bw, tram_routes_nw],
                       ignore_index = True)

train_stops = pd.concat([train_stops_nl, train_stops_ns, train_stops_bw, train_stops_nw],
                         ignore_index=True)
tram_stops = pd.concat([tram_stops_nl, tram_stops_ns, tram_stops_bw, tram_stops_nw],
                         ignore_index=True)
metro_stops = pd.concat([metro_stops_nl, metro_stops_ns, metro_stops_bw, metro_stops_nw],
                         ignore_index=True)
bus_stops = pd.concat([bus_stops_nl, bus_stops_ns, bus_stops_bw, bus_stops_nw],
                         ignore_index=True)

parking_amenities = pd.concat([parking_amenities_nl, parking_amenities_ns, parking_amenities_bw, parking_amenities_nw],
                         ignore_index=True)

traffic_indicators = pd.concat([traffic_indicators_nl, traffic_indicators_ns, traffic_indicators_bw, traffic_indicators_nw],
                         ignore_index=True)

#### Combine public transport stops

In [ ]:
train_stops['transport_type'] = 'train'
tram_stops['transport_type'] = 'tram'
metro_stops['transport_type'] = 'metro'
bus_stops['transport_type'] = 'bus'

public_transport_stops = pd.concat([train_stops, tram_stops,
                                    metro_stops, bus_stops], ignore_index=True)

#### Convert to geodataframe

In [ ]:
metro_routes['geometry'] = metro_routes['geometry'].apply(wkt.loads)
metro_routes_gdf = gpd.GeoDataFrame(metro_routes, geometry='geometry', crs="EPSG:4326")

train_routes['geometry'] = train_routes['geometry'].apply(wkt.loads)
train_routes_gdf = gpd.GeoDataFrame(train_routes, geometry='geometry', crs="EPSG:4326")

bus_routes['geometry'] = bus_routes['geometry'].apply(wkt.loads)
bus_routes_gdf = gpd.GeoDataFrame(bus_routes, geometry='geometry', crs="EPSG:4326")

bike_routes['geometry'] = bike_routes['geometry'].apply(wkt.loads)
bike_routes_gdf = gpd.GeoDataFrame(bike_routes, geometry='geometry', crs="EPSG:4326")

tram_routes['geometry'] = tram_routes['geometry'].apply(wkt.loads)
tram_routes_gdf = gpd.GeoDataFrame(tram_routes, geometry='geometry', crs="EPSG:4326")

train_stops['geometry'] = train_stops['geometry'].apply(wkt.loads)
train_stops_gdf = gpd.GeoDataFrame(train_stops, geometry='geometry', crs="EPSG:4326")

tram_stops['geometry'] = tram_stops['geometry'].apply(wkt.loads)
tram_stops_gdf = gpd.GeoDataFrame(tram_stops, geometry='geometry', crs="EPSG:4326")

metro_stops['geometry'] = metro_stops['geometry'].apply(wkt.loads)
metro_stops_gdf = gpd.GeoDataFrame(metro_stops, geometry='geometry', crs="EPSG:4326")

bus_stops['geometry'] = bus_stops['geometry'].apply(wkt.loads)
bus_stops_gdf = gpd.GeoDataFrame(bus_stops, geometry='geometry', crs="EPSG:4326")

parking_amenities['geometry'] = parking_amenities['geometry'].apply(wkt.loads)
parking_amenities_gdf = gpd.GeoDataFrame(parking_amenities, geometry='geometry', crs="EPSG:4326")

traffic_indicators['geometry'] = traffic_indicators['geometry'].apply(wkt.loads)
traffic_indicators_gdf = gpd.GeoDataFrame(traffic_indicators, geometry='geometry', crs="EPSG:4326")

# GPS data preparation

#### Load GPS data

In [ ]:
# Suppress warnings
# warnings.filterwarnings("ignore")

# Read Data
chunk = pd.read_csv("data/events_uu_feat.csv", chunksize=10000) # prepped event data
events = pd.concat(chunk)

chunk = pd.read_csv("data/geolocations_uu_feat.csv", chunksize=10000) # prepped geolocations data
geolocations = pd.concat(chunk)

# Re-enable warnings
# warnings.filterwarnings("default")

#### Convert geolocations to geodataframe

In [ ]:
# Create a GeoDataFrame for the geolocations data
gps_points = [Point(lon, lat) for lon, lat in zip(geolocations['lon'], geolocations['lat'])]
geolocations_gdf = gpd.GeoDataFrame(geolocations, geometry=gps_points, crs="EPSG:4326")

#### Create event-level linestrings

In [ ]:
# Function to create geometry for each group
def create_geometry(group):
    points = list(zip(group['lon'], group['lat']))
    if len(points) > 1:
        return LineString(points)
    else:
        return Point(points[0])

In [ ]:
# Group by event_id and create geometries
grouped_geolocations = geolocations.groupby('event_id')
geometries = grouped_geolocations.apply(create_geometry, include_groups = False)

# Convert to GeoDataFrame
events_gdf = gpd.GeoDataFrame(geometries, columns=['geometry'], crs="EPSG:4326").reset_index()

# Proximity to public transport stations and parking amenities

#### Compute Haversine great-circle distance between points

In [ ]:
def haversine_distance_vectorized(lat1, lon1, lat2, lon2):

    R = 6371  # Earth radius in kilometers
    
    # Convert latitude and longitude from degrees to radians
    lat1 = np.radians(lat1[:, np.newaxis])  # Shape (n, 1)
    lon1 = np.radians(lon1[:, np.newaxis])  # Shape (n, 1)
    lat2 = np.radians(lat2)                 # Shape (m,)
    lon2 = np.radians(lon2)                 # Shape (m,)

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    return R * c  # Distance in kilometers

#### Compute nearest station/amenity

In [ ]:
def calc_nearest(df_gps, df_osm, label, n1=5, n2=5):
    """
    Calculate the average distance to the nearest parking amenity for the first n1 and last n2 GPS points, respectively.
    """
    results = []

    # Prepare the station coordinates
    station_lats = df_osm['lat'].values
    station_lons = df_osm['lon'].values

    # Loop over each user and track
    for (user_id, event_id), trip_data in df_gps.groupby(['user_id', 'event_id']):
        # Select first and last points
        first_points = trip_data[trip_data['accuracy']<=20].head(n1)
        last_points = trip_data[trip_data['accuracy']<=20].tail(n2)

        # if empty, use the point with lowest accuracy from the first n1*2 observations
        if first_points.empty:
            first_points = trip_data.head(n1*2)
            first_points = first_points[first_points['accuracy']==first_points['accuracy'].min()]
            last_points = trip_data.tail(n2*2)
            last_points = last_points[last_points['accuracy']==last_points['accuracy'].min()]

        # Extract GPS points for the first and last sections
        gps_lats_start = first_points['lat'].values
        gps_lons_start = first_points['lon'].values
        gps_lats_end = last_points['lat'].values
        gps_lons_end = last_points['lon'].values
        
        # Calculate distances for the start points (to nearest station)
        distances_start = haversine_distance_vectorized(gps_lats_start, gps_lons_start, station_lats, station_lons)
        nearest_distances_start = distances_start.min(axis=1)  # Nearest station distance for each point
        avg_distance_start = nearest_distances_start.mean()  # Average distance for the first n1 points

        # Calculate distances for the end points (to nearest station)
        distances_end = haversine_distance_vectorized(gps_lats_end, gps_lons_end, station_lats, station_lons)
        nearest_distances_end = distances_end.min(axis=1)  # Nearest station distance for each point
        avg_distance_end = nearest_distances_end.mean()  # Average distance for the last n2 points

        results.append({
            'event_id': event_id,
            f'dist_to_{label}_start': avg_distance_start,
            f'dist_to_{label}_end': avg_distance_end
        })

    # Convert results to DataFrame
    return pd.DataFrame(results)

#### Proximity to parking

In [ ]:
# filter parking amenity
amenity_parking = parking_amenities[parking_amenities['amenity']=='parking']
amenity_parking = amenity_parking.dropna(subset=['lat', 'lon'])

amenity_parking_prox = calc_nearest(geolocations, amenity_parking, "parking")

#### Proximity to parking entrance

In [ ]:
# filter parking entrance amenity
amenity_parking_entrance = parking_amenities[parking_amenities['amenity']=='parking_entrance']
amenity_parking_entrance = amenity_parking_entrance.dropna(subset=['lat', 'lon'])

amenity_parking_entrance_prox = calc_nearest(geolocations, amenity_parking_entrance, "parking_entrance")

#### Proximity to charging station

In [ ]:
# filter charging station amenity
amenity_charging_station = parking_amenities[parking_amenities['amenity']=='charging_station']
amenity_charging_station = amenity_charging_station.dropna(subset=['lat', 'lon'])

amenity_charging_station_prox = calc_nearest(geolocations, amenity_charging_station, "charging_station")

#### Proximity to bike parking

In [ ]:
amenity_bike_parking = parking_amenities[parking_amenities['amenity']=='bicycle_parking']
amenity_bike_parking = amenity_bike_parking.dropna(subset=['lat', 'lon'])

amenity_bike_parking_prox = calc_nearest(geolocations, amenity_bike_parking, "bike_parking")

#### Proximity to bike rental

In [ ]:
amenity_bike_rental = parking_amenities[parking_amenities['amenity']=='bicycle_rental']
amenity_bike_rental = amenity_bike_rental.dropna(subset=['lat', 'lon'])

amenity_bike_rental_prox = calc_nearest(geolocations, amenity_bike_rental, "bike_rental")

#### Combine and save parking amenities proximity data

In [ ]:
# merge the two dataframes 
parking_amenities_prox = pd.merge(amenity_parking_prox, amenity_parking_entrance_prox, on=['event_id'], how="inner", suffixes=('_df1', '_df2'))
parking_amenities_prox = pd.merge(parking_amenities_prox, amenity_charging_station_prox, on=['event_id'], how="inner", suffixes=('_df1', '_df2'))
parking_amenities_prox = pd.merge(parking_amenities_prox, amenity_bike_parking_prox, on=['event_id'], how="inner", suffixes=('_df1', '_df2'))
parking_amenities_prox = pd.merge(parking_amenities_prox, amenity_bike_rental_prox, on=['event_id'], how="inner", suffixes=('_df1', '_df2'))

# Write the dataframe to a CSV file
parking_amenities_prox.to_csv("osm_parking_amenities_prox_uu.csv", index=False)

#### Proximity to train station

In [ ]:
train_station_prox = calc_nearest(geolocations, train_stops, "train_station")

#### Proximity to tram station

In [ ]:
tram_station_prox = calc_nearest(geolocations, tram_stops, "tram_station")

#### Proximity to bus station

In [ ]:
bus_station_prox = calc_nearest(geolocations, bus_stops, "bus_station")

#### Proximity to metro station

In [ ]:
metro_station_prox = calc_nearest(geolocations, metro_stops, "metro_station")

#### Combine and save public transport station proximity data

In [ ]:
# merge the two dataframes 
public_transport_stations_prox = pd.merge(train_station_prox, tram_station_prox, on=['event_id'], how="inner", suffixes=('_df1', '_df2'))
public_transport_stations_prox = pd.merge(public_transport_stations_prox, bus_station_prox, on=['event_id'], how="inner", suffixes=('_df1', '_df2'))
public_transport_stations_prox = pd.merge(public_transport_stations_prox, metro_station_prox, on=['event_id'], how="inner", suffixes=('_df1', '_df2'))

# Write the dataframe to a CSV file
public_transport_stations_prox.to_csv("osm_public_transport_stations_prox_uu.csv", index=False)

# Count public transport route/station and traffic indicator overlap

#### Create buffer around OSM and GPS geometries

In [ ]:
# add 10m and 20m buffer to GPS data (event-level geometries)
events_gdf = events_gdf.to_crs(epsg=3395)
events_gdf['buffered_geometry_10'] = events_gdf['geometry'].apply(lambda x: x.buffer(10))
events_gdf['buffered_geometry_20'] = events_gdf['geometry'].apply(lambda x: x.buffer(20))

In [ ]:
# add buffer (10m) to OSM public transport (& bike) routes
train_routes_gdf = train_routes_gdf.to_crs(epsg=3395)
train_routes_gdf['buffered_geometry_10'] = train_routes_gdf['geometry'].apply(lambda x: x.buffer(10))
tram_routes_gdf = tram_routes_gdf.to_crs(epsg=3395)
tram_routes_gdf['buffered_geometry_10'] = tram_routes_gdf['geometry'].apply(lambda x: x.buffer(10))
bus_routes_gdf = bus_routes_gdf.to_crs(epsg=3395)
bus_routes_gdf['buffered_geometry_10'] = bus_routes_gdf['geometry'].apply(lambda x: x.buffer(10))
metro_routes_gdf = metro_routes_gdf.to_crs(epsg=3395)
metro_routes_gdf['buffered_geometry_10'] = metro_routes_gdf['geometry'].apply(lambda x: x.buffer(10))
bike_routes_gdf = bike_routes_gdf.to_crs(epsg=3395)
bike_routes_gdf['buffered_geometry_10'] = bike_routes_gdf['geometry'].apply(lambda x: x.buffer(10))

In [ ]:
# add buffer (20m) to OSM public transport stations
train_stops_gdf = train_stops_gdf.to_crs(epsg=3395)
train_stops_gdf['buffered_geometry_20'] = train_stops_gdf['geometry'].apply(lambda x: x.buffer(20))
tram_stops_gdf = tram_stops_gdf.to_crs(epsg=3395)
tram_stops_gdf['buffered_geometry_20'] = tram_stops_gdf['geometry'].apply(lambda x: x.buffer(20))
bus_stops_gdf = bus_stops_gdf.to_crs(epsg=3395)
bus_stops_gdf['buffered_geometry_20'] = bus_stops_gdf['geometry'].apply(lambda x: x.buffer(20))
metro_stops_gdf = metro_stops_gdf.to_crs(epsg=3395)
metro_stops_gdf['buffered_geometry_20'] = metro_stops_gdf['geometry'].apply(lambda x: x.buffer(20))

In [ ]:
# add buffer (10m) to OSM traffic indicators
traffic_indicators_gdf = traffic_indicators_gdf.to_crs(epsg=3395)
traffic_indicators_gdf['buffered_geometry_10'] = traffic_indicators_gdf['geometry'].apply(lambda x: x.buffer(10))

In [ ]:
# get total area from events
events_gdf['total_buffer_area_10'] = events_gdf['buffered_geometry_10'].area
events_gdf['total_buffer_area_20'] = events_gdf['buffered_geometry_20'].area

#### Count intersections with public transport & bike routes

In [ ]:
# Check for train route intersection
joined_train_routes = gpd.sjoin(train_routes_gdf.set_geometry('buffered_geometry_10'),
                         events_gdf.set_geometry('buffered_geometry_10'), 
                         how='inner', 
                         predicate='intersects')

# Create the output DataFrame
matched_train_routes = joined_train_routes[['geometry_left', 'event_id', 'geometry_right']].rename(columns={
    'geometry_left': 'cls_train',
    'geometry_right': 'cls_geometry'
})
matched_train_routes = matched_train_routes.merge(events_gdf[['event_id', 'total_buffer_area_10']], on='event_id', how='left')

events_train_routes_counts = matched_train_routes.groupby('event_id').agg(
    num_train_routes=('cls_train', 'size'),  # Count the number of metro overlaps
    total_buffer_area_10=('total_buffer_area_10', 'first')  # Get the total area for each track
)

# Normalize by total area
events_train_routes_counts['normalized_num_train_routes'] = (events_train_routes_counts['num_train_routes'] / events_train_routes_counts['total_buffer_area_10'])
events_train_routes = events_train_routes_counts.reset_index()
events_train_routes = events_train_routes[['event_id','num_train_routes', 'normalized_num_train_routes' ]]

In [ ]:
# Check for tram route intersection
joined_tram_routes = gpd.sjoin(tram_routes_gdf.set_geometry('buffered_geometry_10'),
                         events_gdf.set_geometry('buffered_geometry_10'), 
                         how='inner', 
                         predicate='intersects')

# Create the output DataFrame
matched_tram_routes = joined_tram_routes[['geometry_left', 'event_id', 'geometry_right']].rename(columns={
    'geometry_left': 'cls_tram',
    'geometry_right': 'cls_geometry'
})
matched_tram_routes = matched_tram_routes.merge(events_gdf[['event_id', 'total_buffer_area_10']], on='event_id', how='left')

events_tram_routes_counts = matched_tram_routes.groupby('event_id').agg(
    num_tram_routes=('cls_tram', 'size'),  # Count the number of metro overlaps
    total_buffer_area_10=('total_buffer_area_10', 'first')  # Get the total area for each track
)

# Normalize by total area
events_tram_routes_counts['normalized_num_tram_routes'] = (events_tram_routes_counts['num_tram_routes'] / events_tram_routes_counts['total_buffer_area_10'])
events_tram_routes = events_tram_routes_counts.reset_index()
events_tram_routes = events_tram_routes[['event_id','num_tram_routes', 'normalized_num_tram_routes' ]]

In [ ]:
# Check for bus route intersection
joined_bus_routes = gpd.sjoin(bus_routes_gdf.set_geometry('buffered_geometry_10'),
                         events_gdf.set_geometry('buffered_geometry_10'), 
                         how='inner', 
                         predicate='intersects')

# Create the output DataFrame
matched_bus_routes = joined_bus_routes[['geometry_left', 'event_id', 'geometry_right']].rename(columns={
    'geometry_left': 'cls_bus',
    'geometry_right': 'cls_geometry'
})
matched_bus_routes = matched_bus_routes.merge(events_gdf[['event_id', 'total_buffer_area_10']], on='event_id', how='left')

events_bus_routes_counts = matched_bus_routes.groupby('event_id').agg(
    num_bus_routes=('cls_bus', 'size'),  # Count the number of metro overlaps
    total_buffer_area_10=('total_buffer_area_10', 'first')  # Get the total area for each track
)

# Normalize by total area
events_bus_routes_counts['normalized_num_bus_routes'] = (events_bus_routes_counts['num_bus_routes'] / events_bus_routes_counts['total_buffer_area_10'])
events_bus_routes = events_bus_routes_counts.reset_index()
events_bus_routes = events_bus_routes[['event_id','num_bus_routes', 'normalized_num_bus_routes' ]]

In [ ]:
# Check for metro route intersection
joined_metro_routes = gpd.sjoin(metro_routes_gdf.set_geometry('buffered_geometry_10'),
                         events_gdf.set_geometry('buffered_geometry_10'), 
                         how='inner', 
                         predicate='intersects')

# Create the output DataFrame
matched_metro_routes = joined_metro_routes[['geometry_left', 'event_id', 'geometry_right']].rename(columns={
    'geometry_left': 'cls_metro',
    'geometry_right': 'cls_geometry'
})
matched_metro_routes = matched_metro_routes.merge(events_gdf[['event_id', 'total_buffer_area_10']], on='event_id', how='left')

events_metro_routes_counts = matched_metro_routes.groupby('event_id').agg(
    num_metro_routes=('cls_metro', 'size'),  # Count the number of metro overlaps
    total_buffer_area_10=('total_buffer_area_10', 'first')  # Get the total area for each track
)

# Normalize by total area
events_metro_routes_counts['normalized_num_metro_routes'] = (events_metro_routes_counts['num_metro_routes'] / events_metro_routes_counts['total_buffer_area_10'])
events_metro_routes = events_metro_routes_counts.reset_index()
events_metro_routes = events_metro_routes[['event_id','num_metro_routes', 'normalized_num_metro_routes' ]]

In [ ]:
# Check for bike route intersection
joined_bike_routes = gpd.sjoin(bike_routes_gdf.set_geometry('buffered_geometry_10'),
                         events_gdf.set_geometry('buffered_geometry_10'), 
                         how='inner', 
                         predicate='intersects')

# Create the output DataFrame
matched_bike_routes = joined_bike_routes[['geometry_left', 'event_id', 'geometry_right']].rename(columns={
    'geometry_left': 'cls_bike',
    'geometry_right': 'cls_geometry'
})
matched_bike_routes = matched_bike_routes.merge(events_gdf[['event_id', 'total_buffer_area_10']], on='event_id', how='left')

events_bike_routes_counts = matched_bike_routes.groupby('event_id').agg(
    num_bike_routes=('cls_bike', 'size'),  # Count the number of metro overlaps
    total_buffer_area_10=('total_buffer_area_10', 'first')  # Get the total area for each track
)

# Normalize by total area
events_bike_routes_counts['normalized_num_bike_routes'] = (events_bike_routes_counts['num_bike_routes'] / events_bike_routes_counts['total_buffer_area_10'])
events_bike_routes = events_bike_routes_counts.reset_index()
events_bike_routes = events_bike_routes[['event_id','num_bike_routes', 'normalized_num_bike_routes' ]]

#### Save public transport route count data

In [ ]:
# Write the dataframe to a CSV file
events_metro_routes.to_csv("osm_metro_route_counts_uu.csv", index=False)
events_train_routes.to_csv("osm_train_route_counts_uu.csv", index=False)
events_tram_routes.to_csv("osm_tram_route_counts_uu.csv", index=False)
events_bus_routes.to_csv("osm_bus_route_counts_uu.csv", index=False)
events_bike_routes.to_csv("osm_bike_route_counts_uu.csv", index=False)

#### Count intersections with public transport stations

In [ ]:
# Check for train station intersection
joined_train_stations = gpd.sjoin(train_stops_gdf.set_geometry('buffered_geometry_20'), 
                                  events_gdf.set_geometry('buffered_geometry_20'), 
                                  how='inner', 
                                  predicate='intersects')

# Create the output DataFrame
matched_train_stations = joined_train_stations[['geometry_left', 'event_id', 'geometry_right']].rename(columns={
    'geometry_left': 'cls_train',
    'geometry_right': 'cls_geometry'
})

matched_train_stations = matched_train_stations.merge(events_gdf[['event_id', 'total_buffer_area_20']], on='event_id', how='left')

events_train_stations_counts = matched_train_stations.groupby('event_id').agg(
    num_train_stations=('cls_train', 'size'),  
    total_buffer_area_20=('total_buffer_area_20', 'first')  # Get the total area for each track
)

# Normalize by total area
events_train_stations_counts['normalized_num_train_stations'] = (events_train_stations_counts['num_train_stations'] / events_train_stations_counts['total_buffer_area_20'])
events_train_stations = events_train_stations_counts.reset_index()
events_train_stations = events_train_stations[['event_id','num_train_stations', 'normalized_num_train_stations' ]]

In [ ]:
# Check for tram station intersection
joined_tram_stations = gpd.sjoin(tram_stops_gdf.set_geometry('buffered_geometry_20'), 
                                  events_gdf.set_geometry('buffered_geometry_20'), 
                                  how='inner', 
                                  predicate='intersects')

# Create the output DataFrame
matched_tram_stations = joined_tram_stations[['geometry_left', 'event_id', 'geometry_right']].rename(columns={
    'geometry_left': 'cls_tram',
    'geometry_right': 'cls_geometry'
})

matched_tram_stations = matched_tram_stations.merge(events_gdf[['event_id', 'total_buffer_area_20']], on='event_id', how='left')

events_tram_stations_counts = matched_tram_stations.groupby('event_id').agg(
    num_tram_stations=('cls_tram', 'size'),  
    total_buffer_area_20=('total_buffer_area_20', 'first')  # Get the total area for each track
)

# Normalize by total area
events_tram_stations_counts['normalized_num_tram_stations'] = (events_tram_stations_counts['num_tram_stations'] / events_tram_stations_counts['total_buffer_area_20'])
events_tram_stations = events_tram_stations_counts.reset_index()
events_tram_stations = events_tram_stations[['event_id','num_tram_stations', 'normalized_num_tram_stations' ]]

In [ ]:
# Check for bus station intersection
joined_bus_stations = gpd.sjoin(bus_stops_gdf.set_geometry('buffered_geometry_20'), 
                                  events_gdf.set_geometry('buffered_geometry_20'), 
                                  how='inner', 
                                  predicate='intersects')

# Create the output DataFrame
matched_bus_stations = joined_bus_stations[['geometry_left', 'event_id', 'geometry_right']].rename(columns={
    'geometry_left': 'cls_bus',
    'geometry_right': 'cls_geometry'
})

matched_bus_stations = matched_bus_stations.merge(events_gdf[['event_id', 'total_buffer_area_20']], on='event_id', how='left')

events_bus_stations_counts = matched_bus_stations.groupby('event_id').agg(
    num_bus_stations=('cls_bus', 'size'),  
    total_buffer_area_20=('total_buffer_area_20', 'first')  # Get the total area for each track
)

# Normalize by total area
events_bus_stations_counts['normalized_num_bus_stations'] = (events_bus_stations_counts['num_bus_stations'] / events_bus_stations_counts['total_buffer_area_20'])
events_bus_stations = events_bus_stations_counts.reset_index()
events_bus_stations = events_bus_stations[['event_id','num_bus_stations', 'normalized_num_bus_stations' ]]

In [ ]:
# Check for metro station intersection
joined_metro_stations = gpd.sjoin(metro_stops_gdf.set_geometry('buffered_geometry_20'), 
                                  events_gdf.set_geometry('buffered_geometry_20'), 
                                  how='inner', 
                                  predicate='intersects')

# Create the output DataFrame
matched_metro_stations = joined_metro_stations[['geometry_left', 'event_id', 'geometry_right']].rename(columns={
    'geometry_left': 'cls_metro',
    'geometry_right': 'cls_geometry'
})

matched_metro_stations = matched_metro_stations.merge(events_gdf[['event_id', 'total_buffer_area_20']], on='event_id', how='left')

events_metro_stations_counts = matched_metro_stations.groupby('event_id').agg(
    num_metro_stations=('cls_metro', 'size'),  
    total_buffer_area_20=('total_buffer_area_20', 'first')  # Get the total area for each track
)

# Normalize by total area
events_metro_stations_counts['normalized_num_metro_stations'] = (events_metro_stations_counts['num_metro_stations'] / events_metro_stations_counts['total_buffer_area_20'])
events_metro_stations = events_metro_stations_counts.reset_index()
events_metro_stations = events_metro_stations[['event_id','num_metro_stations', 'normalized_num_metro_stations' ]]

#### Save public transport station count data

In [ ]:
# Write the dataframe to a CSV file
events_metro_stations.to_csv("osm_metro_station_counts_uu.csv", index=False)
events_train_stations.to_csv("osm_train_station_counts_uu.csv", index=False)
events_tram_stations.to_csv("osm_tram_station_counts_uu.csv", index=False)
events_bus_stations.to_csv("osm_bus_station_counts_uu.csv", index=False)

#### Count intersections with traffic indicators

In [ ]:
# filter osm traffic indicators for crossings
traffic_crossing_gdf = traffic_indicators_gdf[traffic_indicators_gdf['highway']=='crossing']

# check for crossing intersections
joined_traffic_crossing = gpd.sjoin(traffic_crossing_gdf.set_geometry('buffered_geometry_10'), 
                   events_gdf.set_geometry('buffered_geometry_10'), 
                   how='inner', 
                   predicate='intersects')

# Create the output DataFrame
matched_traffic_crossing = joined_traffic_crossing[['geometry_left', 'event_id', 'geometry_right']].rename(columns={
    'geometry_left': 'cls_crossing',
    'geometry_right': 'cls_geometry'
})

matched_traffic_crossing = matched_traffic_crossing.merge(events_gdf[['event_id', 'total_buffer_area_10']], on='event_id', how='left')

events_traffic_crossing_counts = matched_traffic_crossing.groupby('event_id').agg(
    num_crossing=('cls_crossing', 'size'),  # Count the number of junctions
    total_buffer_area_10=('total_buffer_area_10', 'first')  # Get the total area for each track
)

# Normalize by total area
events_traffic_crossing_counts['normalized_num_crossings'] = (events_traffic_crossing_counts['num_crossing'] / events_traffic_crossing_counts['total_buffer_area_10'])
events_traffic_crossing = events_traffic_crossing_counts.reset_index()

events_traffic_crossing = events_traffic_crossing[['event_id','num_crossing', 'normalized_num_crossings' ]]

In [ ]:
# filter osm traffic indicators for motorway junctions
traffic_motorway_junction_gdf = traffic_indicators_gdf[traffic_indicators_gdf['highway']=='motorway_junction']

# check for crossing intersections
joined_traffic_motorway_junction = gpd.sjoin(traffic_motorway_junction_gdf.set_geometry('buffered_geometry_10'), 
                   events_gdf.set_geometry('buffered_geometry_10'), 
                   how='inner', 
                   predicate='intersects')

# Create the output DataFrame
matched_traffic_motorway_junction = joined_traffic_motorway_junction[['geometry_left', 'event_id', 'geometry_right']].rename(columns={
    'geometry_left': 'cls_motorway_junction',
    'geometry_right': 'cls_geometry'
})

matched_traffic_motorway_junction = matched_traffic_motorway_junction.merge(events_gdf[['event_id', 'total_buffer_area_10']], on='event_id', how='left')

events_traffic_motorway_junction_counts = matched_traffic_motorway_junction.groupby('event_id').agg(
    num_motorway_junction=('cls_motorway_junction', 'size'),  # Count the number of junctions
    total_buffer_area_10=('total_buffer_area_10', 'first')  # Get the total area for each track
)

# Normalize by total area
events_traffic_motorway_junction_counts['normalized_num_motorway_junction'] = (events_traffic_motorway_junction_counts['num_motorway_junction'] / events_traffic_motorway_junction_counts['total_buffer_area_10'])
events_traffic_motorway_junction = events_traffic_motorway_junction_counts.reset_index()

events_traffic_motorway_junction = events_traffic_motorway_junction[['event_id','num_motorway_junction', 'normalized_num_motorway_junction' ]]

In [ ]:
# filter osm traffic indicators for traffic signals
traffic_signals_gdf = traffic_indicators_gdf[traffic_indicators_gdf['highway']=='traffic_signals']

# check for crossing intersections
joined_traffic_signals = gpd.sjoin(traffic_signals_gdf.set_geometry('buffered_geometry_10'), 
                   events_gdf.set_geometry('buffered_geometry_10'), 
                   how='inner', 
                   predicate='intersects')

# Create the output DataFrame
matched_traffic_signals = joined_traffic_signals[['geometry_left', 'event_id', 'geometry_right']].rename(columns={
    'geometry_left': 'cls_traffic_signals',
    'geometry_right': 'cls_geometry'
})

matched_traffic_signals = matched_traffic_signals.merge(events_gdf[['event_id', 'total_buffer_area_10']], on='event_id', how='left')

events_traffic_signals_counts = matched_traffic_signals.groupby('event_id').agg(
    num_traffic_signals=('cls_traffic_signals', 'size'),  # Count the number of junctions
    total_buffer_area_10=('total_buffer_area_10', 'first')  # Get the total area for each track
)

# Normalize by total area
events_traffic_signals_counts['normalized_num_traffic_signals'] = (events_traffic_signals_counts['num_traffic_signals'] / events_traffic_signals_counts['total_buffer_area_10'])
events_traffic_signals = events_traffic_signals_counts.reset_index()

events_traffic_signals = events_traffic_signals[['event_id','num_traffic_signals', 'normalized_num_traffic_signals' ]]

In [ ]:
# filter osm traffic indicators for turning circles
traffic_turning_circle_gdf = traffic_indicators_gdf[traffic_indicators_gdf['highway']=='turning_circle']

# check for crossing intersections
joined_traffic_turning_circle = gpd.sjoin(traffic_turning_circle_gdf.set_geometry('buffered_geometry_10'), 
                   events_gdf.set_geometry('buffered_geometry_10'), 
                   how='inner', 
                   predicate='intersects')

# Create the output DataFrame
matched_traffic_turning_circle = joined_traffic_turning_circle[['geometry_left', 'event_id', 'geometry_right']].rename(columns={
    'geometry_left': 'cls_turning_circle',
    'geometry_right': 'cls_geometry'
})

matched_traffic_turning_circle = matched_traffic_turning_circle.merge(events_gdf[['event_id', 'total_buffer_area_10']], on='event_id', how='left')

events_traffic_turning_circle_counts = matched_traffic_turning_circle.groupby('event_id').agg(
    num_turning_circle=('cls_turning_circle', 'size'),  # Count the number of junctions
    total_buffer_area_10=('total_buffer_area_10', 'first')  # Get the total area for each track
)

# Normalize by total area
events_traffic_turning_circle_counts['normalized_num_turning_circle'] = (events_traffic_turning_circle_counts['num_turning_circle'] / events_traffic_turning_circle_counts['total_buffer_area_10'])
events_traffic_turning_circle = events_traffic_turning_circle_counts.reset_index()

events_traffic_turning_circle = events_traffic_turning_circle[['event_id','num_turning_circle', 'normalized_num_turning_circle' ]]

In [ ]:
# filter osm traffic indicators for stops
traffic_stop_gdf = traffic_indicators_gdf[traffic_indicators_gdf['highway']=='stop']

# check for crossing intersections
joined_traffic_stop = gpd.sjoin(traffic_stop_gdf.set_geometry('buffered_geometry_10'), 
                   events_gdf.set_geometry('buffered_geometry_10'), 
                   how='inner', 
                   predicate='intersects')

# Create the output DataFrame
matched_traffic_stop = joined_traffic_stop[['geometry_left', 'event_id', 'geometry_right']].rename(columns={
    'geometry_left': 'cls_stop',
    'geometry_right': 'cls_geometry'
})

matched_traffic_stop = matched_traffic_stop.merge(events_gdf[['event_id', 'total_buffer_area_10']], on='event_id', how='left')

events_traffic_stop_counts = matched_traffic_stop.groupby('event_id').agg(
    num_stop=('cls_stop', 'size'),  # Count the number of junctions
    total_buffer_area_10=('total_buffer_area_10', 'first')  # Get the total area for each track
)

# Normalize by total area
events_traffic_stop_counts['normalized_num_stop'] = (events_traffic_stop_counts['num_stop'] / events_traffic_stop_counts['total_buffer_area_10'])
events_traffic_stop = events_traffic_stop_counts.reset_index()

events_traffic_stop = events_traffic_stop[['event_id','num_stop', 'normalized_num_stop' ]]

In [ ]:
# filter osm traffic indicators for speed cameras
traffic_speed_camera_gdf = traffic_indicators_gdf[traffic_indicators_gdf['highway']=='speed_camera']

# check for crossing intersections
joined_traffic_speed_camera = gpd.sjoin(traffic_speed_camera_gdf.set_geometry('buffered_geometry_10'), 
                   events_gdf.set_geometry('buffered_geometry_10'), 
                   how='inner', 
                   predicate='intersects')

# Create the output DataFrame
matched_traffic_speed_camera = joined_traffic_speed_camera[['geometry_left', 'event_id', 'geometry_right']].rename(columns={
    'geometry_left': 'cls_speed_camera',
    'geometry_right': 'cls_geometry'
})

matched_traffic_speed_camera = matched_traffic_speed_camera.merge(events_gdf[['event_id', 'total_buffer_area_10']], on='event_id', how='left')

events_traffic_speed_camera_counts = matched_traffic_speed_camera.groupby('event_id').agg(
    num_speed_camera=('cls_speed_camera', 'size'),  # Count the number of junctions
    total_buffer_area_10=('total_buffer_area_10', 'first')  # Get the total area for each track
)

# Normalize by total area
events_traffic_speed_camera_counts['normalized_num_speed_camera'] = (events_traffic_speed_camera_counts['num_speed_camera'] / events_traffic_speed_camera_counts['total_buffer_area_10'])
events_traffic_speed_camera = events_traffic_speed_camera_counts.reset_index()

events_traffic_speed_camera = events_traffic_speed_camera[['event_id','num_speed_camera', 'normalized_num_speed_camera' ]]

In [ ]:
# filter osm traffic indicators for mini roundabouts
traffic_mini_roundabout_gdf = traffic_indicators_gdf[traffic_indicators_gdf['highway']=='mini_roundabout']

# check for crossing intersections
joined_traffic_mini_roundabout = gpd.sjoin(traffic_mini_roundabout_gdf.set_geometry('buffered_geometry_10'), 
                   events_gdf.set_geometry('buffered_geometry_10'), 
                   how='inner', 
                   predicate='intersects')

# Create the output DataFrame
matched_traffic_mini_roundabout = joined_traffic_mini_roundabout[['geometry_left', 'event_id', 'geometry_right']].rename(columns={
    'geometry_left': 'cls_mini_roundabout',
    'geometry_right': 'cls_geometry'
})

matched_traffic_mini_roundabout = matched_traffic_mini_roundabout.merge(events_gdf[['event_id', 'total_buffer_area_10']], on='event_id', how='left')

events_traffic_mini_roundabout_counts = matched_traffic_mini_roundabout.groupby('event_id').agg(
    num_mini_roundabout=('cls_mini_roundabout', 'size'),  # Count the number of junctions
    total_buffer_area_10=('total_buffer_area_10', 'first')  # Get the total area for each track
)

# Normalize by total area
events_traffic_mini_roundabout_counts['normalized_num_mini_roundabout'] = (events_traffic_mini_roundabout_counts['num_mini_roundabout'] / events_traffic_mini_roundabout_counts['total_buffer_area_10'])
events_traffic_mini_roundabout = events_traffic_mini_roundabout_counts.reset_index()

events_traffic_mini_roundabout = events_traffic_mini_roundabout[['event_id','num_mini_roundabout', 'normalized_num_mini_roundabout' ]]

In [ ]:
# filter osm traffic indicators for street lamps
traffic_street_lamp_gdf = traffic_indicators_gdf[traffic_indicators_gdf['highway']=='street_lamp']

# check for crossing intersections
joined_traffic_street_lamp = gpd.sjoin(traffic_street_lamp_gdf.set_geometry('buffered_geometry_10'), 
                   events_gdf.set_geometry('buffered_geometry_10'), 
                   how='inner', 
                   predicate='intersects')

# Create the output DataFrame
matched_traffic_street_lamp = joined_traffic_street_lamp[['geometry_left', 'event_id', 'geometry_right']].rename(columns={
    'geometry_left': 'cls_street_lamp',
    'geometry_right': 'cls_geometry'
})

matched_traffic_street_lamp = matched_traffic_street_lamp.merge(events_gdf[['event_id', 'total_buffer_area_10']], on='event_id', how='left')

events_traffic_street_lamp_counts = matched_traffic_street_lamp.groupby('event_id').agg(
    num_street_lamp=('cls_street_lamp', 'size'),  # Count the number of junctions
    total_buffer_area_10=('total_buffer_area_10', 'first')  # Get the total area for each track
)

# Normalize by total area
events_traffic_street_lamp_counts['normalized_num_street_lamp'] = (events_traffic_street_lamp_counts['num_street_lamp'] / events_traffic_street_lamp_counts['total_buffer_area_10'])
events_traffic_street_lamp = events_traffic_street_lamp_counts.reset_index()

events_traffic_street_lamp = events_traffic_street_lamp[['event_id','num_street_lamp', 'normalized_num_street_lamp' ]]

#### Save traffic indicator count data

In [ ]:
# Write the dataframe to a CSV file
events_traffic_crossing.to_csv("osm_traffic_crossing_counts_uu.csv", index=False)
events_traffic_motorway_junction.to_csv("osm_traffic_motorway_junction_counts_uu.csv", index=False)
events_traffic_signals.to_csv("osm_traffic_signals_counts_uu.csv", index=False)
events_traffic_turning_circle.to_csv("osm_traffic_turning_circle_counts_uu.csv", index=False)
events_traffic_stop.to_csv("osm_traffic_stop_counts_uu.csv", index=False)
events_traffic_speed_camera.to_csv("osm_traffic_speed_camera_counts_uu.csv", index=False)
events_traffic_mini_roundabout.to_csv("osm_traffic_mini_roundabout_counts_uu.csv", index=False)
events_traffic_street_lamp.to_csv("osm_traffic_street_lamp_counts_uu.csv", index=False)

# Distance (intersection) to public transport & bike routes

In [ ]:
metro_routes_merged = pd.merge(
    matched_metro_routes, 
    events_gdf, 
    on="event_id", 
    how="inner")

train_routes_merged = pd.merge(
    matched_train_routes, 
    events_gdf, 
    on="event_id", 
    how="inner")

tram_routes_merged = pd.merge(
    matched_tram_routes, 
    events_gdf, 
    on="event_id", 
    how="inner")

bus_routes_merged = pd.merge(
    matched_bus_routes, 
    events_gdf, 
    on="event_id", 
    how="inner")

bike_routes_merged = pd.merge(
    matched_bike_routes, 
    events_gdf, 
    on="event_id", 
    how="inner")

#### Distance to train routes

In [ ]:
# Function to calculate the distance to train track
def calculate_distance_to_train(row):
    cls_geometry = row['cls_geometry']
    cls_train_track = row['cls_train']
    
    # Extract points from the LineString (cls_geometry)
    points = [Point(coord) for coord in cls_geometry.coords]  # Convert each coordinate into a Point
    
    # Calculate the distance from each point to the nearest point on the cls_train_track
    distances = [point.distance(cls_train_track) for point in points]
    
    # Calculate multiple statistics on the distances
    min_distance = min(distances)
    max_distance = max(distances)
    mean_distance = np.mean(distances)
    std_distance = np.std(distances)
    
    # Return the statistics as a dictionary
    return {'min_distance_train': min_distance, 
            'max_distance_train': max_distance, 
            'mean_distance_train': mean_distance, 
            'std_distance_train': std_distance}


def apply_distance_train(chunk):
    # Apply the function and convert the dictionary output into a DataFrame
    results = chunk.apply(calculate_distance_to_train, axis=1)
    return pd.DataFrame(results.tolist(), index=chunk.index)

# Function to apply calculation in parallel
def parallel_calculation_train(df, num_chunks=4):
    # Split the DataFrame into chunks
    chunk_size = len(df) // num_chunks
    chunks = [df.iloc[i:i + chunk_size] for i in range(0, len(df), chunk_size)]
    
    # Initialize Pool and parallelize the calculation
    with Pool(num_chunks) as pool:
        result = pool.map(apply_distance_train, chunks)
    
    # Combine the results
    distances = pd.concat(result, axis=0)
    return distances

In [ ]:
# Start
start_time = time.time()

#parallel calculation
distance_stats_train = parallel_calculation_train(train_routes_merged, num_chunks=1)

# Assign the results to separate columns
train_routes_merged[['min_distance_train', 'max_distance_train', 'mean_distance_train', 'std_distance_train']] = distance_stats_train

# End 
end_time = time.time()

# Output the time taken
elapsed_time = end_time - start_time
print(f"Time taken for parallel processing: {elapsed_time:.2f} seconds")

#### Distance to tram routes

In [ ]:
# Function to calculate the distance to tram track
def calculate_distance_to_tram(row):
    cls_geometry = row['cls_geometry']
    cls_tram_track = row['cls_tram']
    
    # Extract points from the LineString (cls_geometry)
    points = [Point(coord) for coord in cls_geometry.coords]  # Convert each coordinate into a Point
    
    # Calculate the distance from each point to the nearest point on the cls_metro_track
    distances = [point.distance(cls_tram_track) for point in points]
    
    # Calculate multiple statistics on the distances
    min_distance = min(distances)
    max_distance = max(distances)
    mean_distance = np.mean(distances)
    std_distance = np.std(distances)
    
    # Return the statistics as a dictionary
    return {'min_distance_tram': min_distance, 
            'max_distance_tram': max_distance, 
            'mean_distance_tram': mean_distance, 
            'std_distance_tram': std_distance}


def apply_distance_tram(chunk):
    # Apply the function and convert the dictionary output into a DataFrame
    results = chunk.apply(calculate_distance_to_tram, axis=1)
    return pd.DataFrame(results.tolist(), index=chunk.index)

# Function to apply calculation in parallel
def parallel_calculation_tram(df, num_chunks=4):
    # Split the DataFrame into chunks
    chunk_size = len(df) // num_chunks
    chunks = [df.iloc[i:i + chunk_size] for i in range(0, len(df), chunk_size)]
    
    # Initialize Pool and parallelize the calculation
    with Pool(num_chunks) as pool:
        result = pool.map(apply_distance_tram, chunks)
    
    # Combine the results
    distances = pd.concat(result, axis=0)
    return distances

In [ ]:
# Start
start_time = time.time()

#parallel calculation
distance_stats_tram = parallel_calculation_tram(tram_routes_merged, num_chunks=2)

# Assign the results to separate columns
tram_routes_merged[['min_distance_tram', 'max_distance_tram', 'mean_distance_tram', 'std_distance_tram']] = distance_stats_tram

# End 
end_time = time.time()

# Output the time taken
elapsed_time = end_time - start_time
print(f"Time taken for parallel processing: {elapsed_time:.2f} seconds")

#### Distance to bus routes

In [ ]:
# Function to calculate the distance to bus track
def calculate_distance_to_bus(row):
    cls_geometry = row['cls_geometry']
    cls_bus_track = row['cls_bus']
    
    # Extract points from the LineString (cls_geometry)
    points = [Point(coord) for coord in cls_geometry.coords]  # Convert each coordinate into a Point
    
    # Calculate the distance from each point to the nearest point on the cls_metro_track
    distances = [point.distance(cls_bus_track) for point in points]
    
    # Calculate multiple statistics on the distances
    min_distance = min(distances)
    max_distance = max(distances)
    mean_distance = np.mean(distances)
    std_distance = np.std(distances)
    
    # Return the statistics as a dictionary
    return {'min_distance_bus': min_distance, 
            'max_distance_bus': max_distance, 
            'mean_distance_bus': mean_distance, 
            'std_distance_bus': std_distance}


def apply_distance_bus(chunk):
    # Apply the function and convert the dictionary output into a DataFrame
    results = chunk.apply(calculate_distance_to_bus, axis=1)
    return pd.DataFrame(results.tolist(), index=chunk.index)

# Function to apply calculation in parallel
def parallel_calculation_bus(df, num_chunks=4):
    # Split the DataFrame into chunks
    chunk_size = len(df) // num_chunks
    chunks = [df.iloc[i:i + chunk_size] for i in range(0, len(df), chunk_size)]
    
    # Initialize Pool and parallelize the calculation
    with Pool(num_chunks) as pool:
        result = pool.map(apply_distance_bus, chunks)
    
    # Combine the results
    distances = pd.concat(result, axis=0)
    return distances

In [ ]:
# Start
start_time = time.time()

#parallel calculation
distance_stats_bus = parallel_calculation_bus(bus_routes_merged, num_chunks=4)

# Assign the results to separate columns
bus_routes_merged[['min_distance_bus', 'max_distance_bus', 'mean_distance_bus', 'std_distance_bus']] = distance_stats_bus

# End 
end_time = time.time()

# Output the time taken
elapsed_time = end_time - start_time
print(f"Time taken for parallel processing: {elapsed_time:.2f} seconds")

#### Distance to metro routes

In [ ]:

# Function to calculate the distance to metro track
def calculate_distance_to_metro(row):
    cls_geometry = row['cls_geometry']
    cls_metro_track = row['cls_metro']
    
    # Extract points from the LineString (cls_geometry)
    points = [Point(coord) for coord in cls_geometry.coords]  # Convert each coordinate into a Point
    
    # Calculate the distance from each point to the nearest point on the cls_metro_track
    distances = [point.distance(cls_metro_track) for point in points]
    
    # Calculate multiple statistics on the distances
    min_distance = min(distances)
    max_distance = max(distances)
    mean_distance = np.mean(distances)
    std_distance = np.std(distances)
    
    # Return the statistics as a dictionary
    return {'min_distance_metro': min_distance, 
            'max_distance_metro': max_distance, 
            'mean_distance_metro': mean_distance, 
            'std_distance_metro': std_distance}

# Refactor to avoid lambda inside multiprocessing
def apply_distance_metro(chunk):
    # Apply the function and convert the dictionary output into a DataFrame
    results = chunk.apply(calculate_distance_to_metro, axis=1)
    return pd.DataFrame(results.tolist(), index=chunk.index)

# Function to apply calculation in parallel
def parallel_calculation_metro(df, num_chunks=4):
    # Split the DataFrame into chunks
    chunk_size = len(df) // num_chunks
    chunks = [df.iloc[i:i + chunk_size] for i in range(0, len(df), chunk_size)]
    
    # Initialize Pool and parallelize the calculation
    with Pool(num_chunks) as pool:
        result = pool.map(apply_distance_metro, chunks)
    
    # Combine the results
    distances = pd.concat(result, axis=0)
    return distances

In [ ]:
# Start
start_time = time.time()

#parallel calculation
distance_stats_metro = parallel_calculation_metro(metro_routes_merged, num_chunks=1)

# Assign the results to separate columns
metro_routes_merged[['min_distance_metro', 'max_distance_metro', 'mean_distance_metro', 'std_distance_metro']] = distance_stats_metro

# End 
end_time = time.time()

# Output the time taken
elapsed_time = end_time - start_time
print(f"Time taken for parallel processing: {elapsed_time:.2f} seconds")

#### Distance to bike routes

In [ ]:
# Function to calculate the distance to bike track
def calculate_distance_to_bike(row):
    cls_geometry = row['cls_geometry']
    cls_bike_track = row['cls_bike']
    
    # Extract points from the LineString (cls_geometry)
    points = [Point(coord) for coord in cls_geometry.coords]  # Convert each coordinate into a Point
    
    # Calculate the distance from each point to the nearest point on the cls_metro_track
    distances = [point.distance(cls_bike_track) for point in points]
    
    # Calculate multiple statistics on the distances
    min_distance = min(distances)
    max_distance = max(distances)
    mean_distance = np.mean(distances)
    std_distance = np.std(distances)
    
    # Return the statistics as a dictionary
    return {'min_distance_bike': min_distance, 
            'max_distance_bike': max_distance, 
            'mean_distance_bike': mean_distance, 
            'std_distance_bike': std_distance}


def apply_distance_bike(chunk):
    # Apply the function and convert the dictionary output into a DataFrame
    results = chunk.apply(calculate_distance_to_bike, axis=1)
    return pd.DataFrame(results.tolist(), index=chunk.index)

# Function to apply calculation in parallel
def parallel_calculation_bike(df, num_chunks=4):
    # Split the DataFrame into chunks
    chunk_size = len(df) // num_chunks
    chunks = [df.iloc[i:i + chunk_size] for i in range(0, len(df), chunk_size)]
    
    # Initialize Pool and parallelize the calculation
    with Pool(num_chunks) as pool:
        result = pool.map(apply_distance_bike, chunks)
    
    # Combine the results
    distances = pd.concat(result, axis=0)
    return distances

In [ ]:
# Start
start_time = time.time()

#parallel calculation
distance_stats_bike = parallel_calculation_bike(bike_routes_merged, num_chunks=1)

# Assign the results to separate columns
bike_routes_merged[['min_distance_bike', 'max_distance_bike', 'mean_distance_bike', 'std_distance_bike']] = distance_stats_bike

# End 
end_time = time.time()

# Output the time taken
elapsed_time = end_time - start_time
print(f"Time taken for parallel processing: {elapsed_time:.2f} seconds")

#### Save distance to routes data

In [ ]:
# save distance to train routes
events_train_routes_prox = train_routes_merged[['event_id', 'min_distance_train', 'max_distance_train', 'mean_distance_train', 'std_distance_train']]
events_train_routes_prox = events_train_routes_prox.loc[events_train_routes_prox.groupby('event_id')['mean_distance_train'].idxmin()]
events_train_routes_prox = events_train_routes_prox.reset_index(drop=True)

events_train_routes_prox.to_csv("osm_train_route_prox_uu.csv", index=False)

In [ ]:
# save distance to tram routes
events_tram_routes_prox = tram_routes_merged[['event_id', 'min_distance_tram', 'max_distance_tram', 'mean_distance_tram', 'std_distance_tram']]
events_tram_routes_prox = events_tram_routes_prox.loc[events_tram_routes_prox.groupby('event_id')['mean_distance_tram'].idxmin()]
events_tram_routes_prox = events_tram_routes_prox.reset_index(drop=True)

events_tram_routes_prox.to_csv("osm_tram_route_prox_uu.csv", index=False)

In [ ]:
# save distance to bus routes
events_bus_routes_prox = bus_routes_merged[['event_id', 'min_distance_bus', 'max_distance_bus', 'mean_distance_bus', 'std_distance_bus']]
events_bus_routes_prox = events_bus_routes_prox.loc[events_bus_routes_prox.groupby('event_id')['mean_distance_bus'].idxmin()]
events_bus_routes_prox = events_bus_routes_prox.reset_index(drop=True)

events_bus_routes_prox.to_csv("osm_bus_route_prox_uu.csv", index=False)

In [ ]:
# save distance to metro routes
events_metro_routes_prox = metro_routes_merged[['event_id', 'min_distance_metro', 'max_distance_metro', 'mean_distance_metro', 'std_distance_metro']]
events_metro_routes_prox = events_metro_routes_prox.loc[events_metro_routes_prox.groupby('event_id')['mean_distance_metro'].idxmin()]
events_metro_routes_prox = events_metro_routes_prox.reset_index(drop=True)

events_metro_routes_prox.to_csv("osm_metro_route_prox_uu.csv", index=False)

In [ ]:
# save distance to bike routes
events_bike_routes_prox = bike_routes_merged[['event_id', 'min_distance_bike', 'max_distance_bike', 'mean_distance_bike', 'std_distance_bike']]
events_bike_routes_prox = events_bike_routes_prox.loc[events_bike_routes_prox.groupby('event_id')['mean_distance_bike'].idxmin()]
events_bike_routes_prox = events_bike_routes_prox.reset_index(drop=True)

events_bike_routes_prox.to_csv("osm_bike_route_prox_uu.csv", index=False)